In [0]:
dbutils.widgets.text(
    "bronze_path",
    "dbfs:/Volumes/workspace/ecommerce/ecommerce_data/medallion/bronze/events"
)

dbutils.widgets.text(
    "silver_path",
    "dbfs:/Volumes/workspace/ecommerce/ecommerce_data/medallion/silver/events"
)


In [0]:
from pyspark.sql import functions as F

bronze_path = dbutils.widgets.get("bronze_path")
silver_path = dbutils.widgets.get("silver_path")

bronze = spark.read.format("delta").load(bronze_path)

silver = (bronze
    .withColumn("event_time", F.to_timestamp("event_time"))
    .filter(F.col("event_time").isNotNull())
    .filter(F.col("price").isNotNull())
    .filter(F.col("price") > 0)
    .filter(F.col("price") < 10000)
    .dropDuplicates(["user_session", "event_time"])
    .withColumn("event_date", F.to_date("event_time"))
    .withColumn(
        "price_tier",
        F.when(F.col("price") < 10, "budget")
         .when(F.col("price") < 50, "mid")
         .otherwise("premium")
    )
)

silver.write.format("delta").mode("overwrite").save(silver_path)

print("SILVER completed")


SILVER completed
